In [24]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import re
import numpy as np

In [2]:
data = pd.read_json("train_spider.json")

In [3]:
data

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


In [4]:
data['question_toks'][4]

['What',
 'is',
 'the',
 'average',
 'number',
 'of',
 'employees',
 'of',
 'the',
 'departments',
 'whose',
 'rank',
 'is',
 'between',
 '10',
 'and',
 '15',
 '?']

In [5]:
data["sql"][4]

{'from': {'table_units': [['table_unit', 0]], 'conds': []},
 'select': [False, [[5, [0, [0, 6, False], None]]]],
 'where': [[False, 1, [0, [0, 4, False], None], 10.0, 15.0]],
 'groupBy': [],
 'having': [],
 'orderBy': [],
 'limit': None,
 'intersect': None,
 'union': None,
 'except': None}

In [6]:
data_others = pd.read_json('train_others.json')

In [7]:
data_others

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ..."
1,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, popul...","{'from': {'table_units': [['table_unit', 1]], ..."
2,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ..."
3,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",where is the most populated area of wyoming,"[where, is, the, most, populated, area, of, wy...","{'from': {'table_units': [['table_unit', 1]], ..."
4,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",which city in wyoming has the largest population,"[which, city, in, wyoming, has, the, largest, ...","{'from': {'table_units': [['table_unit', 1]], ..."
...,...,...,...,...,...,...,...
1654,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...","[SELECT, t2.house_number, ,, t1.name, FROM, LO...","[select, t2, ., house_number, ,, t1, ., name, ...",where can i find a restaurant in the bay area ?,"[where, can, i, find, a, restaurant, in, the, ...","{'from': {'table_units': [['table_unit', 2], [..."
1655,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...","[SELECT, t2.house_number, ,, t1.name, FROM, LO...","[select, t2, ., house_number, ,, t1, ., name, ...",give me a good restaurant in the bay area ?,"[give, me, a, good, restaurant, in, the, bay, ...","{'from': {'table_units': [['table_unit', 2], [..."
1656,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...","[SELECT, t2.house_number, ,, t1.name, FROM, LO...","[select, t2, ., house_number, ,, t1, ., name, ...",what are some good restaurants in the bay area ?,"[what, are, some, good, restaurants, in, the, ...","{'from': {'table_units': [['table_unit', 2], [..."
1657,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...","[SELECT, t2.house_number, ,, t1.name, FROM, LO...","[select, t2, ., house_number, ,, t1, ., name, ...",give me some good restaurants in the bay area ?,"[give, me, some, good, restaurants, in, the, b...","{'from': {'table_units': [['table_unit', 2], [..."


In [8]:
datadev = pd.read_json("dev.json")

In [9]:
datadev

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","Show name, country, age for all singers ordere...","[Show, name, ,, country, ,, age, for, all, sin...","{'from': {'table_units': [['table_unit', 1]], ..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","What are the names, countries, and ages for ev...","[What, are, the, names, ,, countries, ,, and, ...","{'from': {'table_units': [['table_unit', 1]], ..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","What is the average, minimum, and maximum age ...","[What, is, the, average, ,, minimum, ,, and, m...","{'from': {'table_units': [['table_unit', 1]], ..."
...,...,...,...,...,...,...,...
1029,singer,SELECT Citizenship FROM singer WHERE Birth_Yea...,"[SELECT, Citizenship, FROM, singer, WHERE, Bir...","[select, citizenship, from, singer, where, bir...",What are the citizenships that are shared by s...,"[What, are, the, citizenships, that, are, shar...","{'from': {'table_units': [['table_unit', 0]], ..."
1030,real_estate_properties,SELECT count(*) FROM Other_Available_Features,"[SELECT, count, (, *, ), FROM, Other_Available...","[select, count, (, *, ), from, other_available...",How many available features are there in total?,"[How, many, available, features, are, there, i...","{'from': {'table_units': [['table_unit', 2]], ..."
1031,real_estate_properties,SELECT T2.feature_type_name FROM Other_Availab...,"[SELECT, T2.feature_type_name, FROM, Other_Ava...","[select, t2, ., feature_type_name, from, other...",What is the feature type name of feature AirCon?,"[What, is, the, feature, type, name, of, featu...","{'from': {'table_units': [['table_unit', 2], [..."
1032,real_estate_properties,SELECT T2.property_type_description FROM Prope...,"[SELECT, T2.property_type_description, FROM, P...","[select, t2, ., property_type_description, fro...",Show the property type descriptions of propert...,"[Show, the, property, type, descriptions, of, ...","{'from': {'table_units': [['table_unit', 3], [..."


In [10]:
sampled_data = data.sample(1000, replace=False)
sampled_data.to_json("required_data.json")

In [11]:
sampled_data = pd.read_json("required_data.json")

In [12]:
sampled_data.head(1).transpose()

,2286
db_id,entrepreneur
query,SELECT T2.Weight FROM entrepreneur AS T1 JOIN ...
query_toks,"[SELECT, T2.Weight, FROM, entrepreneur, AS, T1..."
query_toks_no_value,"[select, t2, ., weight, from, entrepreneur, as..."
question,"Return the weights of entrepreneurs, ordered d..."
question_toks,"[Return, the, weights, of, entrepreneurs, ,, o..."
sql,"{'from': {'table_units': [['table_unit', 0], [..."


In [13]:
sampled_data.reset_index(inplace=True)
sampled_data.drop(columns=["index"], inplace=True)

In [14]:
sampled_data["query_toks"][2]

['SELECT',
 'sum',
 '(',
 'T1.salary',
 ')',
 'FROM',
 'salary',
 'AS',
 'T1',
 'JOIN',
 'team',
 'AS',
 'T2',
 'ON',
 'T1.team_id',
 '=',
 'T2.team_id_br',
 'WHERE',
 'T2.name',
 '=',
 "'Boston",
 'Red',
 'Stockings',
 "'",
 'AND',
 'T1.year',
 '=',
 '2010']

In [15]:
sampled_data["query_toks_no_value"][2]

['select',
 'sum',
 '(',
 't1',
 '.',
 'salary',
 ')',
 'from',
 'salary',
 'as',
 't1',
 'join',
 'team',
 'as',
 't2',
 'on',
 't1',
 '.',
 'team_id',
 '=',
 't2',
 '.',
 'team_id_br',
 'where',
 't2',
 '.',
 'name',
 '=',
 'value',
 'and',
 't1',
 '.',
 'year',
 '=',
 'value']

In [16]:
def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

def sub_punct(text: str) -> str:
    """Remove punctuations from the english query string"""
    res = re.sub(r'[^\w\s]', '', text)
    return res

In [18]:
sampled_data["question_toks"]

0      [Who, is, the, advisor, of, Linda, Smith, ?, G...
1      [How, many, lessons, were, taught, by, a, staf...
2      [Which, physicians, prescribe, a, medication, ...
3      [Show, the, names, of, journalists, and, the, ...
4                 [Count, the, number, of, wrestlers, .]
5      [Find, the, names, of, all, the, customers, an...
6      [List, the, name, of, all, different, customer...
7      [List, the, names, of, pilots, that, do, not, ...
8      [Find, the, name, and, college, of, students, ...
9      [Select, the, names, of, manufacturer, whose, ...
10     [Find, the, faculty, rank, that, has, the, lea...
11     [What, are, the, dates, of, publications, in, ...
12     [Count, the, number, of, customers, who, hold,...
13     [List, the, names, of, wrestlers, that, have, ...
14     [Return, the, cities, with, more, than, 3, air...
15     [What, is, the, average, latitude, and, longit...
16     [What, is, the, name, of, the, institution, th...
17     [What, are, the, names, 

In [25]:
sampled_data["punct_removed"] = sampled_data["question"].apply(sub_punct)

In [26]:
sampled_data["punct_removed"]

0      Return the weights of entrepreneurs ordered de...
1      What are the ids for employees who do not work...
2      What is the total salary expenses of team Bost...
3      Show the names and main services for train sta...
4      List all ship names in the order of built year...
5      Return the total number of deaths and total da...
6      Find the average and oldest age for students w...
7        In which year were most departments established
8           What are the course names ordered by credits
9      What are the different first names and highest...
10           List the race class with at least two races
11     What is the total home game attendance of team...
12     How many players did Boston Red Stockings have...
13            What is the lowest and highest rating star
14     What are the names of the airports which are n...
15     What are the ids and names of department store...
16                   Which states have more than 2 parks
17                     List all

In [27]:
sampled_data["whitespace_tokens"] = sampled_data["punct_removed"].apply(whitespace_tokenize)

In [30]:
sampled_data["whitespace_tokens"]

0      [Return, the, weights, of, entrepreneurs, orde...
1      [What, are, the, ids, for, employees, who, do,...
2      [What, is, the, total, salary, expenses, of, t...
3      [Show, the, names, and, main, services, for, t...
4      [List, all, ship, names, in, the, order, of, b...
5      [Return, the, total, number, of, deaths, and, ...
6      [Find, the, average, and, oldest, age, for, st...
7      [In, which, year, were, most, departments, est...
8      [What, are, the, course, names, ordered, by, c...
9      [What, are, the, different, first, names, and,...
10     [List, the, race, class, with, at, least, two,...
11     [What, is, the, total, home, game, attendance,...
12     [How, many, players, did, Boston, Red, Stockin...
13     [What, is, the, lowest, and, highest, rating, ...
14     [What, are, the, names, of, the, airports, whi...
15     [What, are, the, ids, and, names, of, departme...
16           [Which, states, have, more, than, 2, parks]
17              [List, all, cou

In [31]:
sampled_data["query_whitespace_removed"] = sampled_data["query"].apply(whitespace_tokenize)

In [32]:
sampled_data["query_whitespace_removed"]

0      [SELECT, T2.Weight, FROM, entrepreneur, AS, T1...
1      [SELECT, *, FROM, employees, WHERE, department...
2      [SELECT, sum(T1.salary), FROM, salary, AS, T1,...
3      [SELECT, name, ,, main_services, FROM, station...
4      [SELECT, name, FROM, ship, ORDER, BY, built_ye...
5      [SELECT, sum(number_deaths), ,, sum(damage_mil...
6      [SELECT, avg(age), ,, max(age), ,, sex, FROM, ...
7      [SELECT, creation, FROM, department, GROUP, BY...
8      [SELECT, CName, FROM, COURSE, ORDER, BY, Credits]
9      [SELECT, DISTINCT, T2.emp_fname, ,, T3.prof_hi...
10     [SELECT, CLASS, FROM, race, GROUP, BY, CLASS, ...
11     [SELECT, sum(T1.attendance), FROM, home_game, ...
12     [SELECT, count(*), FROM, salary, AS, T1, JOIN,...
13     [SELECT, max(stars), ,, min(stars), FROM, Rating]
14     [SELECT, name, FROM, airport, WHERE, country, ...
15     [SELECT, T2.dept_store_id, ,, T2.store_name, F...
16     [SELECT, state, FROM, park, GROUP, BY, state, ...
17     [SELECT, T1.name, ,, T2.

### a) from the above comparison we can say that there is no difference between the values obtained from stopwords and commonwords removal are same.
### b)we can assume that the data is already in proper tokenized format and is part of question_toks column
### c) we can use the column directly for embeddings now.